In [1]:
# # Use this the first time setting up the conda environment
# !pip install mysql-connector-python
# !pip install pymysql
# !pip install ipython-sql pymysql
# !pip install cryptography
# !pip install PyQt5
import pymysql
import pandas as pd
import hashlib
import csv
import ipywidgets as widgets
from IPython.display import display
import mysql.connector

# Base functions referenced in future cells
def hash_password(password): # kayla update this with your preferred hash
    return hashlib.sha256(password.encode()).hexdigest()

In [2]:
### Fill in your user data
user_info = {
    'username': 'haleyap',
    'password': '421SQLpass!'
}

In [3]:
# Run this cell and it will either connect to your existing database or create it from the sample data
# Connect to the MySQL serve
db = pymysql.connect(
    host='localhost',    
    user=user_info['username'],
    password=user_info['password'],
)

cursor = db.cursor()

try: # try to use database; if so, it's already loaded
    cursor.execute("USE cafeteria")
except: # if not, need to load it in next
    cursor.execute('CREATE DATABASE cafeteria;')
    cursor.execute("USE cafeteria")
    ### Define the students table
    declare_students='''
    CREATE TABLE Students (
        pid INT PRIMARY KEY,
        first_name VARCHAR(255) NOT NULL,
        last_name VARCHAR(255) NOT NULL,
        meal_balance DECIMAL(8, 2) DEFAULT 0.00,
        to_go_boxes_remaining INT DEFAULT 2,
        password_hash CHAR(64) NOT NULL,
        role CHAR(8) DEFAULT "student"
    );
    '''

    ### Define the menu table
    declare_menu='''
    CREATE TABLE Menu (
        item_id INT PRIMARY KEY AUTO_INCREMENT,
        item_name VARCHAR(255) NOT NULL,
        calories_per_serving INT NOT NULL,
        meal_type CHAR(64) NOT NULL,
        available_date DATE NOT NULL,
        price DECIMAL(8, 2) NOT NULL
    );
    '''

    ### Define the transactions table
    declare_transactions='''
    CREATE TABLE Transactions (
        transaction_id INT PRIMARY KEY AUTO_INCREMENT,
        pid INT NOT NULL,
        item_id INT NOT NULL,
        transaction_type VARCHAR(50) NOT NULL,
        transaction_date DATE NOT NULL,
        FOREIGN KEY (pid) REFERENCES Students(pid),
        FOREIGN KEY (item_id) REFERENCES Menu(item_id)
    );
    '''

    # execute and commit changes
    cursor.execute(declare_students)
    cursor.execute(declare_menu)
    cursor.execute(declare_transactions)
    db.commit()

    # load from csv like in HW2

    # Do so for each of the tables
    with open('data/students.csv') as file:
        headers=next(file)
        headers=headers.split(',')
        headers=[h.strip() for h in headers]
        reader=csv.reader(file) 
        info_list=[]
        for r in reader:
            r[-1]=hash_password(r[-1]) # hash password to store
            # I think we weren't supposed to construct the string this way but cannot remember
            cursor.execute(f'INSERT INTO Students ({", ".join(headers)}) VALUES (%s, %s, %s, %s, %s, %s, %s)', r)

    
    with open('data/menu.csv') as file:
        headers=next(file)
        headers=headers.split(',')
        headers=[h.strip() for h in headers]
        reader=csv.reader(file) 
        info_list=[]
        for r in reader:
            # I think we weren't supposed to construct the string this way but cannot remember
            cursor.execute(f'INSERT INTO Menu ({", ".join(headers)}) VALUES (%s, %s, %s, %s, %s, %s)', r)

    with open('data/transactions.csv') as file:
        headers=next(file)
        headers=headers.split(',')
        headers=[h.strip() for h in headers]
        reader=csv.reader(file) 
        info_list=[]
        for r in reader:
            # I think we weren't supposed to construct the string this way but cannot remember
            cursor.execute(f'INSERT INTO Transactions ({", ".join(headers)}) VALUES (%s, %s, %s, %s, %s)', r)
    # Commit changes
    db.commit()

NameError: name 'pymysql' is not defined

In [4]:
# ## Insert Query Example
new_item_details = (6, 'SandwichTest', 600, 'lunch', '2024-11-12', 10.00)
cursor.execute('INSERT INTO Menu (item_id, item_name, calories_per_serving, meal_type, available_date, price) VALUES (%s, %s, %s, %s, %s, %s);', new_item_details)
db.commit()

# ## Select Query Example
cursor.execute('SELECT * FROM Menu')
result = cursor.fetchall()
print(result)

IntegrityError: (1062, "Duplicate entry '5' for key 'menu.PRIMARY'")

In [5]:
### Sign-In Page Backend Code + Queries

test_case='admin'

if test_case=='student':
    entered_pid='730330697'
    entered_password='ramses123'
elif test_case=='admin':
    entered_pid='987654321'
    entered_password='flav0rTown!!!'
elif test_case=='wrong_password':
    entered_pid='987654321'
    entered_password='wrong_pass'
elif test_case=='nonexistant_user':
    entered_pid='222222222'
    entered_password='wrongPass'

cursor.execute('SELECT * FROM Students S WHERE S.pid=%s', entered_pid)
result = cursor.fetchall()

entered_password_hash=hash_password(entered_password)
try:
    if (result[0][5]==entered_password_hash):
        print('Sign-In Success')
        current_user = {
            'pid': result[0][0],
            'first_name': result[0][1],
            'last_name': result[0][2],
            'balance': result[0][3],
            'to_go_remaining': result[0][4],
            'password_hash': result[0][5],
            'role': result[0][6],
        }
        print(current_user)

        if current_user['role']=='admin':
            print('direct to admin page')
        else:
            print('direct to student')
    else:
        print('incorrect password code')
except:
    print('pid not found')


pid not found


In [ ]:
### Landing Page
# based on current user from sign-in cell; 
today_date='2024-11-13'

### Header 
# Common Info
# print(f'Name: {current_user["first_name"]} {current_user["last_name"]}')
print(f'Name: {current_user["first_name"]} {current_user["last_name"]}')
print(f'PID: {current_user["pid"]}')
print(f'{current_user["role"].title()} View')

# Right Header
if current_user["role"]=='student':
    print(f'Account Balance: ${current_user["balance"]}')
    print(f'To-Go Boxes Remaining: {current_user["to_go_remaining"]}/2')
if current_user["role"]=='admin':
    print('Button for New Order')
    print('Button for Student Management')

# Transaction History
if current_user["role"]=='student':
    print("Your Transaction History")
    cursor.execute('SELECT * FROM Transactions T WHERE T.pid=%s', current_user["pid"])
    result = cursor.fetchall()
    print(result)
elif current_user["role"]=='admin':
    print("Transaction History")
    cursor.execute('SELECT * FROM Transactions T')
    result = cursor.fetchall()
    print(result)

# Today's Menu
print("Today's Menu")
cursor.execute('SELECT * FROM Menu M where available_date=%s', today_date)
result = cursor.fetchall()
print(result)




In [5]:
# Create the trigger to prevent negative to-go boxes
prevent_negative_to_go_boxes_trigger = """
CREATE TRIGGER prevent_negative_to_go_boxes
BEFORE UPDATE ON Students
FOR EACH ROW
BEGIN
    IF NEW.to_go_boxes_remaining < 0 THEN
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Error: To-go boxes cannot go negative.';
    END IF;
END;
"""

# Create the trigger to prevent negative meal balance
prevent_negative_meal_balance_trigger = """
CREATE TRIGGER prevent_negative_meal_balance
BEFORE UPDATE ON Students
FOR EACH ROW
BEGIN
    IF NEW.meal_balance < 0 THEN
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Error: Meal balance cannot go negative.';
    END IF;
END;
"""

In [7]:
### Orders Page
today_date='2024-11-13'

# Header showing current student
print(f"Student PID: {current_user['pid']}")

# Create the checkbox widget
to_go_checkbox = widgets.Checkbox(
    value=False,  # Initially unchecked
    description='To-Go Box',
    disabled=False
)

# Display the checkbox
display(to_go_checkbox)

# Function to handle checkbox value
def checkbox_value(change):
    if change['new']:
        print("To-Go Box: Checked")
    else:
        print("To-Go Box: Unchecked")

# Attach the function to the checkbox widget
to_go_checkbox.observe(checkbox_value, names='value')

# Fetch Menu Items for Dropdown
cursor.execute("SELECT item_id, item_name FROM Menu WHERE available_date = %s;", (today_date,))
menu_items = cursor.fetchall()

# Create the Dropdown Menu for Menu Items
dropdown_menu = widgets.Dropdown(
    options=[(item_name, item_id) for item_id, item_name in menu_items],  # Dropdown options
    description='Menu Item:',
    disabled=False
)

# Display the Dropdown menu
display(dropdown_menu)

# Function to handle dropdown selection (automatically adds item to TempOrders table)
def add_item_to_temp_order(change):
    item_id = change['new']
    if item_id is None:
        return  # If no item is selected, do nothing
    
    # Add the selected item to the TempOrders table
    add_to_temp_orders(current_user['pid'], item_id)
    print(f"Added {dict(menu_items)[item_id]} to your order.")

# Attach function to dropdown menu value change
dropdown_menu.observe(add_item_to_temp_order, names='value')

# Create TempOrders table if not exists
create_temp_orders_table = """
CREATE TEMPORARY TABLE IF NOT EXISTS TempOrders (
    temp_id INT AUTO_INCREMENT PRIMARY KEY,
    pid INT NOT NULL,
    item_id INT NOT NULL,
    FOREIGN KEY (pid) REFERENCES Students(pid),
    FOREIGN KEY (item_id) REFERENCES Menu(item_id)
);
"""
cursor.execute(create_temp_orders_table)

# Add items to temporary orders
def add_to_temp_orders(pid, item_id):
    query = "INSERT INTO TempOrders (pid, item_id) VALUES (%s, %s);"
    cursor.execute(query, (pid, item_id))
    db.commit()

# Display temporary orders and calculate total cost
def display_temp_orders(pid):
    query = """
    SELECT M.item_name, M.price
    FROM TempOrders T
    JOIN Menu M ON T.item_id = M.item_id
    WHERE T.pid = %s;
    """
    cursor.execute(query, (pid,))
    temp_orders = cursor.fetchall()
    total_cost = 0
    print("Item Name | Price")
    print("-----------------")
    for item_name, price in temp_orders:
        print(f"{item_name} | ${price:.2f}")
        total_cost += price
    return total_cost

# Display Order Header
print("Order:")
total_cost = display_temp_orders(current_user['pid'])
print(f"Total Cost: ${total_cost:.2f}")

def submit_order(pid, to_go_checked):
    try:
        # Begin transaction
        db.start_transaction()

        # Recalculate total cost
        total_cost = update_temp_orders_display()

        # Validate student's balance and to-go boxes
        cursor.execute("SELECT meal_balance, to_go_boxes_remaining FROM Students WHERE pid = %s;", (pid,))
        student_data = cursor.fetchone()
        meal_balance, to_go_boxes_remaining = student_data

        if to_go_checked and to_go_boxes_remaining <= 0:
            raise Exception("Error: No to-go boxes remaining.")
        if total_cost > meal_balance:
            raise Exception("Error: Insufficient meal balance.")

        # Update Students table
        update_student_query = """
        UPDATE Students
        SET meal_balance = meal_balance - %s,
            to_go_boxes_remaining = to_go_boxes_remaining - %s
        WHERE pid = %s;
        """
        cursor.execute(update_student_query, (total_cost, int(to_go_checked), pid))

        # Insert into Transactions table
        cursor.execute("""
        SELECT item_id FROM TempOrders WHERE pid = %s;
        """, (pid,))
        items = cursor.fetchall()
        for (item_id,) in items:
            insert_transaction_query = """
            INSERT INTO Transactions (pid, item_id, transaction_type, transaction_date)
            VALUES (%s, %s, 'Order', %s);
            """
            cursor.execute(insert_transaction_query, (pid, item_id, today_date))

        # Clear TempOrders
        cursor.execute("DELETE FROM TempOrders WHERE pid = %s;", (pid,))

        # Commit transaction
        db.commit()
        print("Order submitted successfully.")
    except Exception as e:
        # Rollback transaction if any error occurs
        db.rollback()
        print(f"Transaction failed: {str(e)}")


# Submit Button Logic
submit_button = widgets.Button(description="Submit Order")
def on_submit_button_clicked(b):
    submit_order(current_user['pid'], to_go_checkbox.value)

submit_button.on_click(on_submit_button_clicked)
display(submit_button)

# Home Button Logic
def home(pid):
    # Clear TempOrders for this user
    cursor.execute("DELETE FROM TempOrders WHERE pid = %s;", (pid,))
    db.commit()
    print("Returning to Landing Page...")

home_button = widgets.Button(description="Home")
def on_home_button_clicked(b):
    home(current_user['pid'])

home_button.on_click(on_home_button_clicked)
display(home_button)

NameError: name 'current_user' is not defined

In [ ]:
### Be sure to close at the end
### Note that if you close and then try to perform new queries you'll get an InterfaceError: (0, '') error and need to reopen conneciton
db.close()